## **챗 모델과 Message를 사용해 간단한 LLM 애플리케이션 구축하기**

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# os.environ

## 언어 모델 사용하기

LangChain은 다양한 언어 모델을 지원하며, 이들을 서로 교체하여 사용할 수 있습니다.

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", model_provider="openai")
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

ChatModels은 LangChain Runnables의 인스턴스로, 표준화된 인터페이스를 통해 상호작용할 수 있습니다. 모델을 간단히 호출하려면 `.invoke` 메서드에 Messages 목록을 전달하면 됩니다.

## 메시지 (Messages)

메시지(Messages) 는 LangChain에서 모델이 사용하는 컨텍스트의 기본 단위입니다.
이들은 모델의 입력(input) 과 출력(output) 을 나타내며, LLM과의 상호작용에서 대화의 상태(state)를 표현하는 데 필요한 콘텐츠(content) 와 메타데이터(metadata) 를 함께 포함합니다.

메시지는 다음 요소들로 구성됩니다:

- Role (역할)
→ 메시지의 유형을 식별합니다. 예: system, user, assistant 등  
- Content (내용)
→ 메시지의 실제 내용으로, 텍스트뿐만 아니라 이미지, 오디오, 문서 등 다양한 형식을 포함할 수 있습니다.  
- Metadata (메타데이터)
→ 선택적(optional) 필드로, 응답 정보(response info), 메시지 ID, 토큰 사용량(token usage) 등 부가 정보를 담습니다.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

# 메시지 목록을 생성
messages = [
    # 시스템 메시지: 모델에게 수행할 작업이나 역할을 지시합니다.
    SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
    # 사용자 메시지: 사용자가 모델에 보낼 실제 입력 내용입니다.
    HumanMessage("What is LangChain?"),
]

answer = model.invoke(messages)  # `invoke` 메서드를 사용해 모델을 호출합니다.
answer

AIMessage(content='LangChain이 무엇인가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 272, 'prompt_tokens': 40, 'total_tokens': 312, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWYYsEATakrGHWtXbtNwPZVMjrlXm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--cb4b077d-5b7c-427a-a184-1948751bc42d-0', usage_metadata={'input_tokens': 40, 'output_tokens': 272, 'total_tokens': 312, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

In [4]:
answer.pretty_print()

================================== Ai Message ==================================

LangChain이 무엇인가요?


### 텍스트 프롬프트 (Text prompts)

**텍스트 프롬프트(Text prompts)** 는 단순한 **문자열(string)** 형태로 제공됩니다.  
대화 기록(conversation history)을 유지할 필요가 없는 **단순한 생성 작업**(예: 요약, 문장 생성, 번역 등)에 적합합니다.

In [31]:
response = model.invoke("LLM 에 관한 시를 5줄 이내로 지어주세요.")
response.pretty_print()

================================== Ai Message ==================================

데이터 바다 건너온 지혜,
언어로 세상을 엮네.
묻는 자에게 답을 주고,
새로운 이야기를 펼쳐.
코드 속에 피어난 생각의 꽃.


### 딕셔너리 형식 (Dictionary format)

OpenAI의 **Chat Completions 포맷**을 사용해 메시지를 **딕셔너리 형태로 직접 지정**할 수도 있습니다.

In [32]:
# OpenAI 형식
answer = model.invoke([
    {"role": "system", "content": "다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."},
    {"role": "user", "content": "What is LangChain?"},
])
answer.pretty_print()

================================== Ai Message ==================================

랭체인이 무엇인가요?


--------------
## 메시지 유형 (Message types)

* **System message (시스템 메시지)**
  → 모델이 어떻게 행동해야 하는지 지시하고, 상호작용의 **맥락(context)** 을 제공합니다.  
* **Human message (사용자 메시지)**
  → 사용자의 입력을 나타내며, 모델과의 **대화(interaction)** 를 구성합니다.  
* **AI message (AI 메시지)**
  → 모델이 생성한 응답으로, **텍스트 내용(text content)** 뿐 아니라
  **도구 호출(tool calls)** 및 **메타데이터(metadata)** 를 포함할 수 있습니다.  
* **Tool message (도구 메시지)**
  → 모델이 호출한 **도구의 실행 결과(outputs)** 를 나타냅니다.

In [33]:
from langchain.messages import SystemMessage, HumanMessage

system_msg = SystemMessage("""
당신은 웹 프레임워크에 전문성을 가진 시니어 Python 개발자입니다.
설명은 간결하게 5줄 이내로 설명하세요.
""")

messages = [
    system_msg,
    HumanMessage("REST API를 어떻게 만들 수 있나요?")
]
response = model.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

Python에서는 Flask나 Django REST Framework(DRF) 같은 웹 프레임워크를 활용합니다.
리소스 기반의 URL을 설계하고, GET, POST, PUT, DELETE 등 HTTP 메서드를 적절히 매핑하세요.
데이터는 주로 JSON 형식으로 직렬화하여 클라이언트와 주고받습니다.
프레임워크의 라우팅, 뷰, 모델 기능을 사용하여 API 엔드포인트를 구현합니다.
복잡한 API에는 DRF와 같은 전문 라이브러리 사용을 권장합니다.


### AI 메시지 (AI Message)

**AIMessage** 는 **모델 호출(model invocation)** 의 **출력 결과**를 나타냅니다.
이 메시지에는 다음과 같은 요소들을 포함할 수 있습니다:

* **멀티모달 데이터 (Multimodal data)** — 텍스트뿐만 아니라 이미지, 오디오 등의 출력
* **도구 호출 (Tool calls)** — 모델이 외부 도구를 실행한 기록
* **제공자별 메타데이터 (Provider-specific metadata)** — 모델 제공자(OpenAI, Anthropic 등)에 따라 추가로 제공되는 정보

이러한 메타데이터는 나중에 접근하거나 분석할 수 있습니다.

In [13]:
response = model.invoke("AI에 대해서 한줄로 설명해줘.")
print(type(response))  
response

<class 'langchain_core.messages.ai.AIMessage'>


AIMessage(content='인간의 지능을 모방하여 학습하고 추론하며 문제를 해결하는 기술.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--7813ec8f-0a37-4c09-ac0d-9094ab78f6ec-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1421, 'total_tokens': 1432, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1402}})

In [34]:
from langchain.messages import AIMessage, SystemMessage, HumanMessage

# AI 메시지를 수동으로 생성 (예: 대화 기록에 추가하기 위해)
ai_msg = AIMessage("그 질문에 기꺼이 도움을 드리겠습니다!")

# 대화 기록에 추가
messages = [
    SystemMessage("당신은 도움이 되는 어시스턴트입니다."),
    HumanMessage("저를 도와주실 수 있나요?"),
    ai_msg,  # 모델이 이전에 응답한 것처럼 삽입
    HumanMessage("좋아요! 그럼 2 + 2는 얼마인가요?")
]

response = model.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

2 + 2는 **4**입니다.


### 도구 호출 (Tool calls)

모델이 **도구 호출(tool call)** 을 수행할 때, 그 호출 정보는 **AIMessage** 객체 내부에 포함됩니다.

In [18]:
# 도구 정의
def get_weather(location: str) -> str:
    """특정 위치의 날씨를 가져옵니다."""
    ...

# 모델에 도구 바인딩
model_with_tools = model.bind_tools([get_weather])

# 모델 호출
response = model_with_tools.invoke("파리의 날씨는 어때?")

# 모델이 호출한 도구 정보 출력
for tool_call in response.tool_calls:
    print(f"도구 이름: {tool_call['name']}")
    print(f"인자값(Args): {tool_call['args']}")
    print(f"ID: {tool_call['id']}")

도구 이름: get_weather
인자값(Args): {'location': 'Paris'}
ID: e6ce9cfc-af97-4574-86fa-ad423cb163f2


### 토큰 사용량 (Token usage)

**AIMessage** 객체는 `usage_metadata` 필드에
**토큰 사용량(token counts)** 및 기타 **사용 관련 메타데이터(metadata)** 를 저장할 수 있습니다.

In [19]:
response = model.invoke("Hello!")
response.usage_metadata

{'input_tokens': 3,
 'output_tokens': 35,
 'total_tokens': 38,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 25}}

### batch interface 이용 여러개의 메시지 일괄 처리

In [20]:
# 메시지 목록을 생성
batch_messages = [
    [
        SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
        HumanMessage("What is LangChain?")
    ],
    [
        SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
        HumanMessage("How does LangChain work?")
    ],
    [
        SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
        HumanMessage("What are the key features of LangChain?")
    ]
]

# `model.batch()`을 사용하여 여러 개의 메시지를 한 번에 처리
# LangChain은 각 입력을 독립된 invoke() 호출처럼 처리
answers = model.batch(batch_messages)
answers

[AIMessage(content='LangChain은 무엇인가요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--bc578f65-9ff5-4cf3-b691-d80a4d0ba33c-0', usage_metadata={'input_tokens': 27, 'output_tokens': 142, 'total_tokens': 169, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 135}}),
 AIMessage(content='LangChain은 어떻게 작동하나요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--eda465e1-e490-44ca-907a-6f6cdfe39f05-0', usage_metadata={'input_tokens': 28, 'output_tokens': 8, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='LangChain의 주요

In [21]:
# 결과 출력
for idx, ans in enumerate(answers):
    print(f"번역 {idx + 1}: {ans.content}")

번역 1: LangChain은 무엇인가요?
번역 2: LangChain은 어떻게 작동하나요?
번역 3: LangChain의 주요 기능은 무엇인가요?


In [24]:
chunks = []
full_message = None
for chunk in model.stream("LangChain의 주요 기능은 무엇인가요?"):
    chunks.append(chunk)
    print(chunk.text)
    full_message = chunk if full_message is None else full_message + chunk

LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 오픈소스 프레임워크입니다. LLM의 잠재력을 최대한 발휘하고, 복잡한 워크플로우
를 쉽게 구축할 수 있도록 다양한 모듈과 기능을 제공합니다.

LangChain의 주요 기능은 다음과 같습니다:

1.  **LLM 모델 통합 (LLM Integrations):**
    *   OpenAI, Hugging
 Face, Anthropic, Google Gemini 등 다양한 LLM 제공업체 및 로컬에서 호스팅되는 모델과의 쉬운 연동을 지원합니다.
    *   개발자는 특정 LLM에 종속되지 않고, 필요에 따라 모델
을 쉽게 교체할 수 있습니다.

2.  **프롬프트 관리 (Prompt Management):**
    *   프롬프트 템플릿: LLM에 전달할 프롬프트를 동적으로 생성하고 관리
할 수 있는 템플릿 기능을 제공합니다.
    *   프롬프트 엔지니어링: Few-shot 프롬프팅, 출력 파서 등을 통해 LLM의 입력 및 출력을 효율적으로 관리하고 최
적화합니다.

3.  **체인 (Chains):**
    *   여러 구성 요소를 순차적으로 연결하여 복잡한 작업을 수행할 수 있도록 합니다.
    *   예를 들어, 사용자 입력
 -> LLM 호출 -> 결과 파싱 -> 다음 LLM 호출 등으로 이어지는 워크플로우를 만들 수 있습니다.
    *   다양한 미리 정의된 체인(예: `LLMChain`, `SimpleSequential
Chain`)을 제공하며, 사용자가 직접 커스텀 체인을 구성할 수도 있습니다.

4.  **에이전트 (Agents):**
    *   LLM이 스스로 판단하여 어떤 도구를 사용하고 어떤 작업을 수행할
지 결정하도록 돕습니다.
    *   LLM에게 추론 능력과 행동 능력을 부여하여 동적으로 문제를 해결하게 합니다. 예를 들어, "오늘 날씨는 어때? 그리고 내일 주식 시장은 어떻게
 될 것 같아?" 같은 복합적인 질문에 대해 LLM이 적절한 

In [25]:
full_message

AIMessageChunk(content='LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 오픈소스 프레임워크입니다. LLM의 잠재력을 최대한 발휘하고, 복잡한 워크플로우를 쉽게 구축할 수 있도록 다양한 모듈과 기능을 제공합니다.\n\nLangChain의 주요 기능은 다음과 같습니다:\n\n1.  **LLM 모델 통합 (LLM Integrations):**\n    *   OpenAI, Hugging Face, Anthropic, Google Gemini 등 다양한 LLM 제공업체 및 로컬에서 호스팅되는 모델과의 쉬운 연동을 지원합니다.\n    *   개발자는 특정 LLM에 종속되지 않고, 필요에 따라 모델을 쉽게 교체할 수 있습니다.\n\n2.  **프롬프트 관리 (Prompt Management):**\n    *   프롬프트 템플릿: LLM에 전달할 프롬프트를 동적으로 생성하고 관리할 수 있는 템플릿 기능을 제공합니다.\n    *   프롬프트 엔지니어링: Few-shot 프롬프팅, 출력 파서 등을 통해 LLM의 입력 및 출력을 효율적으로 관리하고 최적화합니다.\n\n3.  **체인 (Chains):**\n    *   여러 구성 요소를 순차적으로 연결하여 복잡한 작업을 수행할 수 있도록 합니다.\n    *   예를 들어, 사용자 입력 -> LLM 호출 -> 결과 파싱 -> 다음 LLM 호출 등으로 이어지는 워크플로우를 만들 수 있습니다.\n    *   다양한 미리 정의된 체인(예: `LLMChain`, `SimpleSequentialChain`)을 제공하며, 사용자가 직접 커스텀 체인을 구성할 수도 있습니다.\n\n4.  **에이전트 (Agents):**\n    *   LLM이 스스로 판단하여 어떤 도구를 사용하고 어떤 작업을 수행할지 결정하도록 돕습니다.\n    *   LLM에게 추론 능력과 행동 능력을 부여하여 동적으로 문제를 해결하게 합니다. 예를 들어, "오늘 날씨는 어때? 그리고 내일 주식 시장은 어떻게

### 도구 메시지 (Tool Message)

도구 호출(tool calling)을 지원하는 모델의 경우,
**AI 메시지(AIMessage)** 안에 **도구 호출 정보(tool calls)** 가 포함될 수 있습니다.

**Tool Message(도구 메시지)** 는
**단일 도구 실행의 결과(result)** 를 모델에게 다시 전달하기 위해 사용됩니다.

또한, 도구는 직접 **`ToolMessage` 객체**를 생성하여
실행 결과를 LangChain 에이전트나 모델로 반환할 수 있습니다.

In [29]:
from langchain.messages import ToolMessage

# 모델이 도구 호출을 수행한 후
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",                   # 호출된 도구 이름
        "args": {"location": "San Francisco"},   # 도구에 전달된 인자
        "id": "call_123"                         # 도구 호출 ID
    }]
)

# 도구 실행 후 결과 메시지 생성
weather_result = "맑음, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # 반드시 호출 ID와 일치해야 함
)

# 대화 이어가기
messages = [
    HumanMessage("샌프란시스코의 날씨는 어때?"),
    ai_message,     # 모델이 도구 호출을 요청한 메시지
    tool_message,   # 도구 실행 결과를 모델에 전달
]

response = model.invoke(messages)  # 모델이 도구 결과를 반영하여 후속 응답 생성
response.pretty_print()

================================== Ai Message ==================================

샌프란시스코의 날씨는 **맑고 72°F (약 22°C)**입니다.
